In [1]:
import pyspark

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('earthquake-data-cleaning').getOrCreate()

25/07/24 19:57:47 WARN Utils: Your hostname, recurSe resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/24 19:57:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/07/24 19:57:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('earthquake-data.csv')

In [4]:
df.show()

+--------------------+-------------+---------+----------+------------+-----+-----+-------+----+----------+
|               place|         time|magnitude|       lat|        long|depth|alert|tsunami|  tz|      type|
+--------------------+-------------+---------+----------+------------+-----+-----+-------+----+----------+
|8km SSW of Lytle ...|-631157391770|     2.58|34.1911667|    -117.522| 4.49| null|      0|null|earthquake|
|24km WNW of Searl...|-631215832260|     2.01|35.8593333|-117.6506667|  0.0| null|      0|null|earthquake|
|28km N of El Sauz...|-631241139690|      3.3|32.1433333|-116.6288333|  6.0| null|      0|null|earthquake|
|1km SSW of Artesi...|-631251141040|     1.83|33.8561667|-118.0893333| 0.25| null|      0|null|earthquake|
|16km SE of Primo ...|-631284369930|     3.02|    32.113|-116.8063333|  6.0| null|      0|null|earthquake|
|south of the Fiji...|-631286334600|     null|   -26.927|    -176.566| 15.0| null|      0|null|earthquake|
|10 km SSW of Sawa...|-631292588380| 

In [6]:
df.printSchema()

root
 |-- place: string (nullable = true)
 |-- time: long (nullable = true)
 |-- magnitude: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- depth: double (nullable = true)
 |-- alert: string (nullable = true)
 |-- tsunami: integer (nullable = true)
 |-- tz: double (nullable = true)
 |-- type: string (nullable = true)



In [15]:
df.filter(df.depth.isNull()).show()

+--------------------+-------------+---------+-----+-------+-----+-----+-------+----+----------+
|               place|         time|magnitude|  lat|   long|depth|alert|tsunami|  tz|      type|
+--------------------+-------------+---------+-----+-------+-----+-----+-------+----+----------+
|13 km SSE of Sain...|-636327022000|      4.7| 37.0| -113.5| null| null|      0|null|earthquake|
|6 km NNW of Manil...|-636759044000|      4.5| 40.9| -124.2| null| null|      0|null|earthquake|
|22 km NE of Soled...|-637208080000|      4.7|36.58|-121.17| null| null|      0|null|earthquake|
|4 km S of Chester...|-642568304000|      4.5|40.27|-121.23| null| null|      0|null|earthquake|
|0 km NW of East R...|-643726797000|      3.3|37.95|-122.32| null| null|      0|null|earthquake|
|              Nevada|-645524935000|      4.5| 39.7| -119.8| null| null|      0|null|earthquake|
|11 km WNW of Oak ...|-647357069000|      4.5| 35.8| -121.1| null| null|      0|null|earthquake|
|17 km ENE of Hona...|-6521794

In [16]:
df.na.fill({'depth': 0}).show()

+--------------------+-------------+---------+----------+------------+-----+-----+-------+----+----------+
|               place|         time|magnitude|       lat|        long|depth|alert|tsunami|  tz|      type|
+--------------------+-------------+---------+----------+------------+-----+-----+-------+----+----------+
|8km SSW of Lytle ...|-631157391770|     2.58|34.1911667|    -117.522| 4.49| null|      0|null|earthquake|
|24km WNW of Searl...|-631215832260|     2.01|35.8593333|-117.6506667|  0.0| null|      0|null|earthquake|
|28km N of El Sauz...|-631241139690|      3.3|32.1433333|-116.6288333|  6.0| null|      0|null|earthquake|
|1km SSW of Artesi...|-631251141040|     1.83|33.8561667|-118.0893333| 0.25| null|      0|null|earthquake|
|16km SE of Primo ...|-631284369930|     3.02|    32.113|-116.8063333|  6.0| null|      0|null|earthquake|
|south of the Fiji...|-631286334600|     null|   -26.927|    -176.566| 15.0| null|      0|null|earthquake|
|10 km SSW of Sawa...|-631292588380| 